See [2-select-GD1-stars.ipynb](2-select-GD1-stars.ipynb) to see details about the selection.

In [ ]:
from os import path

# Third-party
import astropy.coordinates as coord
from astropy.table import Table, vstack
from astropy.io import fits
import astropy.units as u
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

from pyia import GaiaData
import gala.coordinates as gc

In [ ]:
# plt.style.use('gd1_paper')
plt.rcParams['font.size'] = 20.0

Load data:

In [ ]:
g = GaiaData('../data/gd1-with-masks.fits')
g = g[g.phi2 > -8*u.deg]

In [ ]:
pm_mask = g.pm_mask
cmd_mask = g.gi_cmd_mask

In [ ]:
(pm_mask & cmd_mask).sum()

Load isochrone and proper motion selection polygons:

In [ ]:
iso = np.load('../output/iso_shifted.npy')
cmd_poly = np.load('../output/iso_poly.npy')
pm_poly = np.load('../output/pm_poly.npy')

In [ ]:
polystyle = dict(color='tab:orange', alpha=0.3, linewidth=0)
ptstyle = dict(color='k', marker='o', linewidth=0, 
               markersize=2, alpha=0.15, rasterized=True)

In [ ]:
phi1_lim = (-90, 10)
phi2_lim = (-6.5, 5.5)

pm1_lim = (-15, 5)
pm2_lim = (-10, 10)

gi_lim = (-0.1, 1.1)
g_lim = (21.2, 16.8)

## Features

In [ ]:
phi1 = g.phi1
phi2 = g.phi2
pm1 = g.pm_phi1_cosphi2_no_reflex
pm2 = g.pm_phi2_no_reflex

In [ ]:
titles = ['Stream', 'Stream', 'Spur', 'Off spur', 'Stream', 'Blob', 'Off blob', 'Off blob 2']

# phi1_min = np.array([-64, -37, -37, -37, -20, -20, -20, -20])*u.deg
# phi1_max = np.array([-56, -29, -29, -29, -12, -12, -12, -12])*u.deg
phi1_min = np.array([-64, -37, -37, -37, -19, -19, -19, -19])*u.deg
phi1_max = np.array([-56, -29, -29, -29, -11, -11, -11, -11])*u.deg
phi2_min = np.array([-1.7, -0.4, 0.5, -1.9, -0.6, -2, 0.3, 1.7])*u.deg
phi2_max = np.array([0.3, 0.5, 2, -0.4, 0.3, -0.6, 1.7, 3.1])*u.deg
Npos = np.size(phi1_min)

pos_masks = []
for i in range(Npos):
    pos_masks = pos_masks + [((g.phi1<phi1_max[i]) & (g.phi1>phi1_min[i]) &
            (g.phi2<phi2_max[i]) & (g.phi2>phi2_min[i]))]

In [ ]:
outer_grid = mpl.gridspec.GridSpec(3, 1, wspace=0.0, hspace=0.35)
top_row = mpl.gridspec.GridSpecFromSubplotSpec(1, 6, subplot_spec=outer_grid[0], wspace=0.5, hspace=0.05,
                                              width_ratios=[1,2,1,2,1,2])
middle_row = mpl.gridspec.GridSpecFromSubplotSpec(1, 1, subplot_spec=outer_grid[1], wspace=0.35, hspace=0.05)
bottom_row = mpl.gridspec.GridSpecFromSubplotSpec(1, 6, subplot_spec=outer_grid[2], wspace=0.5, hspace=0.05,
                                                 width_ratios=[1,2,1,2,1,2])

fig = plt.figure(figsize=(18,12))
ax = plt.Subplot(fig, middle_row[0])
ax_main = fig.add_subplot(ax)

plt.plot(phi1[pm_mask & cmd_mask], phi2[pm_mask & cmd_mask],
         'ko', ms=2, alpha=0.5, rasterized=True)
plt.gca().set_aspect('equal')

plt.xlabel('$\phi_1$ [deg]')
plt.ylabel('$\phi_2$ [deg]')
plt.xlim(phi1_lim)
plt.ylim(phi2_lim)

rows = [top_row, bottom_row]
positions = [[1,2,3], [4,5,6]]
cmaps = [mpl.cm.Purples_r, mpl.cm.Blues_r]
ci = [0.1, 0.4, 0.7]

axes = []

for er, row in enumerate(rows):
    for e, i in enumerate(positions[er]):
        plt.sca(ax_main)
        rect_xy_ = [phi1_min[i].to(u.deg).value, phi2_min[i].to(u.deg).value]
        rect_w_ = phi1_max[i].to(u.deg).value - phi1_min[i].to(u.deg).value
        rect_h_ = phi2_max[i].to(u.deg).value - phi2_min[i].to(u.deg).value
        rect_c_ = cmaps[er](ci[e])
        
        pr = mpl.patches.Rectangle(rect_xy_, rect_w_, rect_h_, ec=rect_c_, lw=2, color=rect_c_, alpha=0.2)
        plt.gca().add_artist(pr)
        
        ax = plt.Subplot(fig, row[2*e])
        ax_1 = fig.add_subplot(ax)
        pos1 = ax_1.get_position()
        
        plt.plot(g.g0[pos_masks[i] & pm_mask] - g.i0[pos_masks[i] & pm_mask], 
                 g.g0[pos_masks[i] & pm_mask], 'ko', ms=1, alpha=0.5)

        pm = mpl.patches.Polygon(cmd_poly, **polystyle)
        plt.gca().add_artist(pm)

        plt.xlim(gi_lim)
        plt.ylim(g_lim)
        plt.xlabel('$(g - i)_0$')
        plt.ylabel('$g_0$')
        

        ax = plt.Subplot(fig, row[2*e+1])
        ax_2 = fig.add_subplot(ax)
        pos2 = ax_2.get_position()

        plt.plot(pm1[pos_masks[i] & cmd_mask].to(u.mas/u.yr), 
                 pm2[pos_masks[i] & cmd_mask].to(u.mas/u.yr),  'ko', ms=1, alpha=0.5)

        # pr = mpl.patches.Rectangle(rect_xy, rect_w, rect_h, color='orange', alpha=0.3)
        pr = mpl.patches.Polygon(pm_poly, **polystyle)
        plt.gca().add_artist(pr)
        
        x1 = pos1.x0
        y1 = pos1.y1
        w = pos2.x1 - pos1.x0
        fig.patches.extend([plt.Rectangle((x1,y1), w, 0.007,
                                  fill=True, color=rect_c_, alpha=0.5, zorder=1000,
                                  transform=fig.transFigure, figure=fig)])

        plt.xlim(-12,12)
        plt.ylim(-12,12)
        plt.xlabel('$\mu_{\phi_1}$ [mas yr$^{-1}$]')
        plt.ylabel('$\mu_{\phi_2}$ [mas yr$^{-1}$]', labelpad=-15)
        plt.title(titles[i], fontsize='medium', position=(-0., 1.05))
        
        if e==1:
            if er==0:
                xy_text_1 = [rect_xy_[0], rect_xy_[1] + rect_h_]
                xy_text_2 = [rect_xy_[0] + rect_w_, rect_xy_[1] + rect_h_]
                xy_coord_1 = [0,0]
                xy_coord_2 = [1,0]
            else:
                xy_text_1 = [rect_xy_[0], rect_xy_[1]]
                xy_text_2 = [rect_xy_[0] + rect_w_, rect_xy_[1]]
                xy_coord_1 = [0,1]
                xy_coord_2 = [1,1]
                
            ax_main.annotate('',xy=xy_coord_1, xytext=xy_text_1, xycoords=ax_1.transAxes, 
                             textcoords=ax_main.transData,
                            arrowprops=dict(color=rect_c_, arrowstyle='-', alpha=0.2, lw=3))
            ax_main.annotate('',xy=xy_coord_2, xytext=xy_text_2, xycoords=ax_2.transAxes, 
                             textcoords=ax_main.transData,
                            arrowprops=dict(color=rect_c_, arrowstyle='-', alpha=0.2, lw=3))

# plt.savefig('../paper/features.pdf')